In [96]:
import pandas as pd

archivo_csv = r'C:/Users/paula/OneDrive/Documentos/Facultad/Analítica en Transporte y Logística/Base de Pedidos Jul DP.csv'
# Mostrar todas las columnas con sus respectivos valores
pd.set_option('display.max_columns', None) 
df = pd.read_csv(archivo_csv)

print(df.head())


                                      id_pedido  id_repartidor  id_comercio  \
0  CqKySjLZSPQfFGdSPDQ01Gd3WzGbHk5iIujhu7WotQs=       88066494     16935120   
1  QEPB2aNxAWT8DIwCyHSRy/ePTrITvNG+87WHgOq3x+U=      106968606     50656494   
2  8p3Eu3YcKuYC1qOC7N0r9XLN1ElsfdJRHILagmY0Szk=       57727242    108476616   
3  lXKSw2EdYahTXfzdJvxSMj6Us1Kt+BxUal1lZoHGaWI=       66504660     40701822   
4  h/qOgjj5OPwS3LJQZz0mqJn9+58upYZHFnMEH5fVb2Y=       80618550     14319516   

              creacion_pedido transicion  transicion_id_repartidor  \
0  2024-07-06 21:48:24.656755  cancelled                88066494.0   
1  2024-07-03 11:09:28.149972  cancelled               106968606.0   
2  2024-07-07 21:02:03.193195  cancelled                57727242.0   
3  2024-07-09 13:23:05.160588  cancelled                66504660.0   
4  2024-07-11 22:56:56.293719  cancelled                80618550.0   

          creacion_transicion  transicion_coordenada_latitud  \
0  2024-07-06 22:14:00.956661           

#### Cantidad de registros en la base de datos vs. la cantidad de pedidos

In [97]:
# Cantidad total de registros
total_registros = len(df)

# Cantidad de valores distintos en la columna 'id_pedido'
id_pedido_unicos = df['id_pedido'].nunique()

# Mostrar los resultados
print(f"Cantidad total de registros: {total_registros}")
print(f"Cantidad de 'id_pedido' distintos: {id_pedido_unicos}")

Cantidad total de registros: 3155642
Cantidad de 'id_pedido' distintos: 323850


In [98]:
# Contar la cantidad de IDs de repartidores distintos
repartidores_unicos = df['id_repartidor'].nunique()
print(f"Cantidad de repartidores distintos: {repartidores_unicos}")

Cantidad de repartidores distintos: 5761


In [99]:
# Contar las categorías únicas en la columna 'transicion'
categorias_transicion = df['transicion'].value_counts()

# Mostrar la cantidad de categorías y cuántas veces aparece cada una
print(f"Cantidad de categorías: {df['transicion'].nunique()}")
print(categorias_transicion)


Cantidad de categorías: 11
transicion
queued              665451
courier_notified    523009
accepted            339727
near_pickup         333634
picked_up           323850
near_dropoff        323407
left_pickup         322706
completed           322450
cancelled             1400
pending                  7
scheduled                1
Name: count, dtype: int64


#### Ordenamos los registros por fecha de transaccion

In [100]:
# Asegúrate de que la columna 'creacion_transicion' esté en formato de fecha y hora
df['creacion_transicion'] = pd.to_datetime(df['creacion_transicion'])

# Ordenar el DataFrame por 'creacion_transicion' en orden cronológico
df = df.sort_values(by='creacion_transicion', ascending=True)

# Imprimir los primeros registros para verificar
print(df)

                                            id_pedido  id_repartidor  \
556106   tBMHeU17rwPedjQxWV5JhFCo02OqhElPMsevJyHmxMU=       56877906   
535077   TSNXj53m+DLUCz7pJpPJmUtR5AtMNdM8g4u1+Cf4KW4=       12704436   
1887076  iSdc/QiV5eDFvPWh8CoOiQfAev9LgQFDq/hLDSP/I7U=      124406310   
2015523  GMqHHWdMh0XDc9YwjDY6yjDJSJFQiYmB09tN1ovCtvI=       85584534   
1877494  mzlqOzwDoX3fv2E2uPDMIo+YgpABTzbCEWCSOJmqfk8=       70267848   
...                                               ...            ...   
240732   5pf0HKIAT5GITMlDNT+9MCSjqjCf6A7tFSlNyckyhl0=      101779710   
2956646  cfVMa6affI06U3kiVFKEBiKhGp2oTwVwWVpbFvM+VT4=       26308776   
1933261  cfVMa6affI06U3kiVFKEBiKhGp2oTwVwWVpbFvM+VT4=       26308776   
2175130  WcnTnCYOYP1ghSAPwgbFklcQx35sAYjjd3CFMgG3hws=       70540812   
1927719  WcnTnCYOYP1ghSAPwgbFklcQx35sAYjjd3CFMgG3hws=       70540812   

         id_comercio             creacion_pedido    transicion  \
556106     129533028  2024-06-30 21:00:09.048317        queued   
535

#### Tasa de rechazos

In [101]:
# Calcular la cantidad de registros donde 'transicion' es 'courier_notified'
total_courier_notified = (df['transicion'] == 'courier_notified').sum()

# Calcular la tasa de rechazos
tasa_rechazos = (df['pre_accept_undispatch'] > 0).sum() / total_courier_notified

# Mostrar el resultado
print(f"La tasa de rechazos es del: {tasa_rechazos}")

La tasa de rechazos es del: 0.34056584112319294


#### Tasa de reasignacion

In [102]:
# Calcular la cantidad de registros donde 'transicion' es 'accepted'
total_accepted = (df['transicion'] == 'accepted').sum()

# Calcular la tasa de reasignaciones
tasa_reasignaciones = (df['post_accept_undispatchs'] > 0).sum() / total_accepted

# Mostrar el resultado
print(f"La tasa de reasignaciones es: {tasa_reasignaciones}")

La tasa de reasignaciones es: 0.04592805399629703


#### TOP pedidos con mayor cantidad de rechazos

In [103]:
# Filtrar los registros donde 'pre_accept_undispatch' o 'post_accept_undispatchs' sean iguales a 1
filtro = df[(df['pre_accept_undispatch'] == 1) | (df['post_accept_undispatchs'] == 1)]

# Contar por 'id_pedido' la cantidad de veces que el pedido tiene un 1 en 'pre_accept_undispatch'
conteo_pre_accept = filtro.groupby('id_pedido')['pre_accept_undispatch'].sum().reset_index(name='conteo_pre_accept')

# Contar por 'id_pedido' la cantidad de veces que el pedido tiene un 1 en 'post_accept_undispatchs'
conteo_post_accept = filtro.groupby('id_pedido')['post_accept_undispatchs'].sum().reset_index(name='conteo_post_accept')

# Unir ambos conteos en una misma tabla
conteo_combinado = pd.merge(conteo_pre_accept, conteo_post_accept, on='id_pedido', how='outer').fillna(0)

# Ordenar la tabla en orden descendente por 'conteo_pre_accept'
conteo_combinado_ordenado = conteo_combinado.sort_values(by='conteo_pre_accept', ascending=False)

# Mostrar la tabla ordenada
print(conteo_combinado_ordenado)

                                          id_pedido  conteo_pre_accept  \
23927  Ew1h9eFx9xGKxizi+19lHRwWxl1o/HjUWwoXlOnGd1Q=                 47   
10594  5SYmrD6KHDMjnrV8s2sYMRegKYwDrf29b/5PLo6zGco=                 43   
44339  TP/weM9adFz/6Cv/NXcFgw3YhTeonwl3YO3EigSrCXg=                 39   
7127   35+T7MN6eSwnP+BCI/mvHbT3fvPq72FWJ3ZQDfub6WQ=                 36   
40410  QUo64m1wl94bxr9NLmAIO830+Fp+3NU3lSu1xri+rYg=                 36   
...                                             ...                ...   
75711  pfjgEGVsPssgMzAT2KiiEmpvBMCjgSa+1R9byEN8ajE=                  0   
67582  jvm+XM9h525zy95OfGRCYb712DzxByoLQYNASP+WVMc=                  0   
46929  VD2bOx9VZ7Q8sqWNk21ifm0mQniaIukrs8fNxKxytiA=                  0   
67580  jvZh8aJvx+JuJ+Wk6yMbVrrY6+2P0HzrJXGdAt0sIjU=                  0   
30294  JPhCPcTHsWqN6/n7vMni0NkAncm2J8Y8516q/XGKewc=                  0   

       conteo_post_accept  
23927                   4  
10594                   0  
44339                   1  

#### Calculo las distancias de los repartidores a los restaurantes pero primero borro los NA

In [104]:
# Contar cuántos registros tienen transicion_coordenada_latitud como NaN
registros_sin_latitud = df['transicion_coordenada_latitud'].isna().sum()
registros_sin_repartidor = df['transicion_id_repartidor'].isna().sum()
registros_sin_longitud = df['transicion_coordenada_longitud'].isna().sum()

# Mostrar el resultado
print(f"Cantidad de registros con transicion_coordenada_latitud = NaN: {registros_sin_latitud}")
print(f"Cantidad de registros con transicion_coordenada_longitud = NaN: {registros_sin_longitud}")
print(f"Cantidad de registros con transicion_id_repartidor = NaN: {registros_sin_repartidor}")


Cantidad de registros con transicion_coordenada_latitud = NaN: 357361
Cantidad de registros con transicion_coordenada_longitud = NaN: 357361
Cantidad de registros con transicion_id_repartidor = NaN: 323865


In [105]:
# Eliminar todos los registros que tienen NaN en las columnas especificadas
df_limpio = df.dropna(subset=['transicion_coordenada_latitud', 'transicion_coordenada_longitud', 'transicion_id_repartidor'])

# Verificar cuántos registros quedan después de eliminar los NaN
print(f"Cantidad de registros después de eliminar NaN: {len(df_limpio)}")


Cantidad de registros después de eliminar NaN: 2798281


In [106]:
# Verificar si hay NaN en alguna columna del DataFrame limpio
nan_restantes = df_limpio.isna().any()

# Mostrar el resultado
print("¿Hay NaN en alguna columna?")
print(nan_restantes)

# Si quieres ver la cantidad de NaN por columna en caso de que haya:
cantidad_nan_por_columna = df_limpio.isna().sum()
print("\nCantidad de NaN por columna:")
print(cantidad_nan_por_columna)

¿Hay NaN en alguna columna?
id_pedido                         False
id_repartidor                     False
id_comercio                       False
creacion_pedido                   False
transicion                        False
transicion_id_repartidor          False
creacion_transicion               False
transicion_coordenada_latitud     False
transicion_coordenada_longitud    False
tiempo_estimado_entrega           False
tiempo_aceptacion_orden           False
distancia_retiro                  False
distancia_entrega                 False
ubicacion_cliente_longitud        False
ubicacion_cliente_latitud         False
ubicacion_comercio_longitud       False
ubicacion_comercio_latitud        False
pre_accept_undispatch             False
post_accept_undispatchs           False
pago_base                         False
pago_rendimiento                  False
pago_incentivos                   False
dtype: bool

Cantidad de NaN por columna:
id_pedido                         0
id_repartidor 

#### Calculo distancias de TODOS LOS repartidores a los comercios (hasta los que rechazan)

In [107]:
import numpy as np

# Función para calcular la distancia Haversine
def haversine(lat1, lon1, lat2, lon2):
    # Convertir las coordenadas de grados a radianes
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Diferencias entre las coordenadas
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Fórmula de Haversine
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Radio de la Tierra en kilómetros
    r = 6371
    
    # Calcular la distancia
    return c * r

# Crear la nueva columna 'distancia_repartidor_comercio' con la distancia entre el repartidor y el comercio
df_limpio['distancia_repartidor_comercio'] = df_limpio.apply(
    lambda row: haversine(row['transicion_coordenada_latitud'], row['transicion_coordenada_longitud'], 
                          row['ubicacion_comercio_latitud'], row['ubicacion_comercio_longitud']), axis=1)

# Verificar los primeros registros para ver la nueva columna
df_limpio[['transicion_coordenada_latitud', 'transicion_coordenada_longitud', 
    'ubicacion_comercio_latitud', 'ubicacion_comercio_longitud', 'distancia_repartidor_comercio']].head()


C:\Users\paula\AppData\Local\Temp\ipykernel_24456\852214146.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['distancia_repartidor_comercio'] = df_limpio.apply(


,transicion_coordenada_latitud,transicion_coordenada_longitud,ubicacion_comercio_latitud,ubicacion_comercio_longitud,distancia_repartidor_comercio
3113034,-8.414142,-19.582787,-19.587054,-8.413354,1729.157310
605163,-8.414142,-19.582787,-19.587054,-8.413354,1729.157310
139004,-8.427104,-19.585368,-19.589170,-8.433020,1726.986917
2746165,-8.430442,-19.581964,-19.589495,-8.433296,1726.464648
1129994,-8.430442,-19.581964,-19.589495,-8.433296,1726.464648


In [108]:
# Crear la nueva columna 'distancia_pedido_total' sumando 'distancia_repartidor_comercio' y 'distancia_entrega'
df_limpio['distancia_pedido_total'] = df_limpio['distancia_repartidor_comercio'] + df_limpio['distancia_entrega']

# Verificar los primeros registros para ver la nueva columna
df_limpio[['distancia_repartidor_comercio', 'distancia_entrega', 'distancia_pedido_total']].head()

C:\Users\paula\AppData\Local\Temp\ipykernel_24456\2522539363.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['distancia_pedido_total'] = df_limpio['distancia_repartidor_comercio'] + df_limpio['distancia_entrega']


,distancia_repartidor_comercio,distancia_entrega,distancia_pedido_total
3113034,1729.157310,418,2147.157310
605163,1729.157310,418,2147.157310
139004,1726.986917,1344,3070.986917
2746165,1726.464648,5060,6786.464648
1129994,1726.464648,5060,6786.464648


#### Agrego una columna para poder identificar los registros donde fueron aceptados

In [109]:
# Crear la columna 'aceptado' donde será 1 si 'transicion' es 'accepted', de lo contrario 0
df_limpio['aceptado'] = df_limpio['transicion'].apply(lambda x: 1 if x == 'accepted' else 0)

C:\Users\paula\AppData\Local\Temp\ipykernel_24456\2021009360.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['aceptado'] = df_limpio['transicion'].apply(lambda x: 1 if x == 'accepted' else 0)


#### Creo una columna para ver los minutos hasta que sea entregada la comida en restaurante

In [110]:
import pandas as pd

# Asegurarse de que ambas columnas son del tipo datetime
df_limpio['tiempo_estimado_entrega'] = pd.to_datetime(df_limpio['tiempo_estimado_entrega'], errors='coerce')
df_limpio['creacion_transicion'] = pd.to_datetime(df_limpio['creacion_transicion'], errors='coerce')

# Crear la columna 'tiempo_hasta_entrega' que calcule la diferencia en minutos
df_limpio['tiempo_hasta_entrega'] = (df_limpio['tiempo_estimado_entrega'] - df_limpio['creacion_transicion']).dt.total_seconds() / 60

# Reemplazar los valores negativos por 0
df_limpio['tiempo_hasta_entrega'] = df_limpio['tiempo_hasta_entrega'].apply(lambda x: max(x, 0))

# Verificar los primeros registros para asegurarse de que la columna se ha creado correctamente
print(df_limpio[['tiempo_estimado_entrega', 'creacion_transicion', 'tiempo_hasta_entrega']].head())

# Verificar cuántos valores NaN hay en la columna 'tiempo_hasta_entrega'
valores_nan = df_limpio['tiempo_hasta_entrega'].isna().sum()

# Verificar cuántos valores negativos hay en la columna 'tiempo_hasta_entrega' (debería ser 0 después de la transformación)
valores_negativos = (df_limpio['tiempo_hasta_entrega'] < 0).sum()

# Mostrar los resultados
print(f"Cantidad de valores NaN en 'tiempo_hasta_entrega': {valores_nan}")
print(f"Cantidad de valores negativos en 'tiempo_hasta_entrega' (después de la transformación): {valores_negativos}")



C:\Users\paula\AppData\Local\Temp\ipykernel_24456\2157252164.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['tiempo_estimado_entrega'] = pd.to_datetime(df_limpio['tiempo_estimado_entrega'], errors='coerce')
C:\Users\paula\AppData\Local\Temp\ipykernel_24456\2157252164.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['creacion_transicion'] = pd.to_datetime(df_limpio['creacion_transicion'], errors='coerce')
C:\Users\paula\AppData\Local\Temp\ipykernel_24456\2157252164.py:8: SettingWi

        tiempo_estimado_entrega        creacion_transicion  \
3113034     2024-06-30 21:26:08 2024-06-30 21:02:10.867048   
605163      2024-06-30 21:26:08 2024-06-30 21:02:13.445229   
139004      2024-06-30 21:30:08 2024-06-30 21:02:28.411115   
2746165     2024-06-30 21:50:07 2024-06-30 21:02:36.768363   
1129994     2024-06-30 21:50:07 2024-06-30 21:02:42.685328   

         tiempo_hasta_entrega  
3113034             23.952216  
605163              23.909246  
139004              27.659815  
2746165             47.503861  
1129994             47.405245  
Cantidad de valores NaN en 'tiempo_hasta_entrega': 8650
Cantidad de valores negativos en 'tiempo_hasta_entrega' (después de la transformación): 0


C:\Users\paula\AppData\Local\Temp\ipykernel_24456\2157252164.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['tiempo_hasta_entrega'] = df_limpio['tiempo_hasta_entrega'].apply(lambda x: max(x, 0))


#### Me doy cuenta que la columna de tiempo_estimado_entrega tiene NaT, borro esos registros

In [111]:
# Revisar los registros donde 'tiempo_hasta_entrega' es NaN
registros_nan = df_limpio[df_limpio['tiempo_hasta_entrega'].isna()]

# Mostrar estos registros para ver posibles problemas en las fechas
print(registros_nan)


                                            id_pedido  id_repartidor  \
1806808  o4pmGPI4zcx6UP3jId15JDCI5HF+fE95C6Ag2SVElis=       84921990   
343338   o4pmGPI4zcx6UP3jId15JDCI5HF+fE95C6Ag2SVElis=       84921990   
1481393  o4pmGPI4zcx6UP3jId15JDCI5HF+fE95C6Ag2SVElis=       84921990   
1694988  o4pmGPI4zcx6UP3jId15JDCI5HF+fE95C6Ag2SVElis=       84921990   
2260865  o4pmGPI4zcx6UP3jId15JDCI5HF+fE95C6Ag2SVElis=       84921990   
...                                               ...            ...   
1246617  ULQbjbyoe3EbcXa4qQL/HWzlNWtE8SthxliTs5p3xpg=       64312434   
2916584  F6gw9v9R5ogVq7yBwt7aJ4j3M2uG5kv6bd7in4hDYDU=       40851462   
1142343  ULQbjbyoe3EbcXa4qQL/HWzlNWtE8SthxliTs5p3xpg=       64312434   
2424994  F6gw9v9R5ogVq7yBwt7aJ4j3M2uG5kv6bd7in4hDYDU=       40851462   
229119   F6gw9v9R5ogVq7yBwt7aJ4j3M2uG5kv6bd7in4hDYDU=       40851462   

         id_comercio             creacion_pedido        transicion  \
1806808     98384430  2024-06-30 21:29:17.560261  courier_notifie

In [112]:
# Eliminar los registros que tienen NaT en 'creacion_transicion' o 'tiempo_estimado_entrega'
df_limpio = df_limpio.dropna(subset=['tiempo_estimado_entrega'])

# Verificar cuántos registros quedan después de eliminar los NaT
print(f"Cantidad de registros después de eliminar NaT: {len(df_limpio)}")

# Verificar si todavía quedan valores NaT en esas columnas
valores_nat_tiempo_estimado = df_limpio['tiempo_estimado_entrega'].isna().sum()

print(f"Cantidad de NaT restantes en 'tiempo_estimado_entrega': {valores_nat_tiempo_estimado}")


Cantidad de registros después de eliminar NaT: 2789631
Cantidad de NaT restantes en 'tiempo_estimado_entrega': 0


#### Uno los datos de los REPARTIDORES a df_limpio

In [113]:
import pandas as pd

archivo_csv = r'C:/Users/paula/OneDrive/Documentos/Facultad/Analítica en Transporte y Logística/Base Repartidores - Dana y Paula.csv'
# Mostrar todas las columnas con sus respectivos valores
pd.set_option('display.max_columns', None) 
repartidores = pd.read_csv(archivo_csv)

print(repartidores.head())
print(repartidores.shape)


   id_repartidor vehiculo  tasa_aceptacion  tasa_cancelacion  \
0      110529264        B             0.86          0.000000   
1      101437602        B             0.99          0.020779   
2       44148960        B             0.89          0.021978   
3      121852368        B             0.94          0.000000   
4       39412596        A             1.00          0.000000   

   pedidos_entregados  antiguedad_semanas  tiempo_medio_entrega  
0                  36           28.731765                 12.24  
1                 377           42.282444                 11.89  
2                 178           58.312500                 15.00  
3                  13            9.795714                 15.90  
4                   1          155.570000                 11.65  
(8731, 7)


In [114]:
# Verificar si hay duplicados en la columna 'id_repartidor'
duplicados = repartidores[repartidores.duplicated(subset='id_repartidor', keep=False)]

# Imprimir los repartidores duplicados (si existen)
if not duplicados.empty:
    print("Hay repartidores duplicados:")
    print(duplicados)
else:
    print("No hay repartidores duplicados.")

Hay repartidores duplicados:
      id_repartidor vehiculo  tasa_aceptacion  tasa_cancelacion  \
4          39412596        A             1.00               0.0   
7         131863026        B             0.80               0.0   
8         111382986        A             1.00               0.0   
25         15242382        A             0.50               0.0   
33         36629550        B             1.00               0.0   
...             ...      ...              ...               ...   
8709      129479880        A             1.00               0.0   
8710      129409704        A             1.00               0.0   
8721      132283308        A             0.33               0.0   
8725      132485322        A             1.00               0.0   
8729       66861990        A             0.50               0.0   

      pedidos_entregados  antiguedad_semanas  tiempo_medio_entrega  
4                      1          155.570000                 11.65  
7                     23    

In [115]:
# Encontrar los repartidores duplicados
duplicados_id = repartidores[repartidores.duplicated(subset='id_repartidor', keep=False)]

# Ordenar los repartidores duplicados por 'id_repartidor' para que queden uno debajo del otro
duplicados_ordenados = duplicados_id.sort_values(by='id_repartidor')

# Imprimir los repartidores duplicados ordenados
print(duplicados_ordenados)

      id_repartidor vehiculo  tasa_aceptacion  tasa_cancelacion  \
6255        7583394        A             1.00          0.000000   
2509        7583394        B             1.00          0.090909   
249         8289024        B             0.99          0.020790   
2087        8289024        A             1.00          0.014851   
2437        9483564        A             0.90          0.013793   
...             ...      ...              ...               ...   
5717      133467012        A             1.00          0.000000   
4852      133471398        A             1.00          0.000000   
4828      133471398        B             0.75          0.000000   
6745      133475526        B             1.00          0.000000   
7429      133475526        A             1.00          0.000000   

      pedidos_entregados  antiguedad_semanas  tiempo_medio_entrega  
6255                  13          245.841111                 12.29  
2509                  10          238.261667             

#### Como estan duplicados los repartidores, los agrupo y actualizo sus valores en las variables

In [116]:
# Agrupar por 'id_repartidor' y aplicar las reglas para combinar los duplicados
repartidores_agrupados = repartidores.groupby('id_repartidor').agg({
    'vehiculo': lambda x: x.loc[x.idxmax()],  # Tomar el vehículo con más pedidos_entregados
    'tasa_aceptacion': 'mean',  # Promedio de las tasas de aceptación
    'tasa_cancelacion': 'mean',  # Promedio de las tasas de cancelación
    'pedidos_entregados': 'sum',  # Sumar los pedidos entregados
    'antiguedad_semanas': 'mean',  # Promedio de antigüedad
    'tiempo_medio_entrega': 'mean'  # Promedio del tiempo medio de entrega
}).reset_index()

# Verificar la cantidad que hay de repartidores
print(repartidores_agrupados.shape)

(8290, 7)


#### Joineo la tabla de PEDIDOS ORIGINAL + REPARTIDORES

In [118]:
# Realizar un merge entre df_limpio y repartidores_agrupados basados en 'transicion_id_repartidor' y 'id_repartidor'
df_limpio_con_repartidores = df_limpio.merge(repartidores_agrupados, left_on='transicion_id_repartidor', right_on='id_repartidor', how='left')

# Verificar los primeros registros para asegurarse de que los datos se han unido correctamente
print(df_limpio_con_repartidores.head())
print(df_limpio_con_repartidores.columns)

                                      id_pedido  id_repartidor_x  id_comercio  \
0  iSdc/QiV5eDFvPWh8CoOiQfAev9LgQFDq/hLDSP/I7U=        124406310     98613018   
1  iSdc/QiV5eDFvPWh8CoOiQfAev9LgQFDq/hLDSP/I7U=        124406310     98613018   
2  tBMHeU17rwPedjQxWV5JhFCo02OqhElPMsevJyHmxMU=         56877906    129533028   
3  mzlqOzwDoX3fv2E2uPDMIo+YgpABTzbCEWCSOJmqfk8=         70267848    121192920   
4  mzlqOzwDoX3fv2E2uPDMIo+YgpABTzbCEWCSOJmqfk8=         70267848    121192920   

              creacion_pedido        transicion  transicion_id_repartidor  \
0  2024-06-30 21:00:09.257475  courier_notified               124406310.0   
1  2024-06-30 21:00:09.257475          accepted               124406310.0   
2  2024-06-30 21:00:09.048317            queued                69517842.0   
3  2024-06-30 21:00:13.517017  courier_notified                70267848.0   
4  2024-06-30 21:00:13.517017          accepted                70267848.0   

         creacion_transicion  transicion_coordenad

#### Unir datos de COMERCIOS

In [121]:
import pandas as pd

archivo_csv = r'C:/Users/paula/OneDrive/Documentos/Facultad/Analítica en Transporte y Logística/Base de Comercios DP.csv'
# Mostrar todas las columnas con sus respectivos valores
pd.set_option('display.max_columns', None) 
comercios = pd.read_csv(archivo_csv)

print(comercios.head())
print(comercios.shape)

   id_comercio    rating  espera_en_vendor
0     43449264  4.111111          3.442593
1    107152818  3.333333          8.122222
2     16394094  3.333333          2.494444
3     37582344  8.500000         25.591667
4    121631262  4.008264          5.453719
(2569, 3)


#### Verifico duplicados y NAs

In [129]:
# Verificar si hay duplicados en la columna 'id_comercio'
comercios_duplicados = comercios[comercios.duplicated(subset='id_comercio', keep=False)]

# Mostrar los comercios duplicados
print(comercios_duplicados)

# Verificar si hay NaN en alguna columna de la tabla de comercios
nas_por_columna = comercios.isna().sum()

# Mostrar la cantidad de NaN por columna
print("Cantidad de NaN por columna:")
print(nas_por_columna)

# Filtrar los registros donde 'espera_en_vendor' es NaN
comercios_con_na = comercios[comercios['espera_en_vendor'].isna()]

# Extraer los 'id_comercio' que tienen NaN en 'espera_en_vendor'
id_comercios_con_na = comercios_con_na['id_comercio']

# Mostrar los id_comercio con NaN
print("ID de comercios con NaN en espera_en_vendor:")
print(id_comercios_con_na)

# Definir los id_comercios que quieres verificar
comercios_a_verificar = [8176278, 126079698]

# Filtrar los registros en 'df_limpio_con_repartidores' que tengan esos id_comercios
cantidad_comercios = df_limpio_con_repartidores[df_limpio_con_repartidores['id_comercio'].isin(comercios_a_verificar)].shape[0]

# Mostrar la cantidad de registros encontrados
print(f"Cantidad de registros con id_comercio 8176278 o 126079698: {cantidad_comercios}")

# Definir los id_comercios que quieres eliminar
comercios_a_eliminar = [8176278, 126079698]

# Eliminar los registros que tengan esos id_comercios
df_limpio_con_repartidores = df_limpio_con_repartidores[~df_limpio_con_repartidores['id_comercio'].isin(comercios_a_eliminar)]

# Verificar la cantidad de registros después de eliminar
print(f"Cantidad de registros después de eliminar: {len(df_limpio_con_repartidores)}")

# Eliminar los registros que tengan esos id_comercios
comercios = comercios[~comercios['id_comercio'].isin(comercios_a_eliminar)]

# Verificar la cantidad de registros después de eliminar
print(f"Cantidad de registros en la tabla de comercios después de eliminar: {len(comercios)}")


Empty DataFrame
Columns: [id_comercio, rating, espera_en_vendor]
Index: []
Cantidad de NaN por columna:
id_comercio         0
rating              0
espera_en_vendor    2
dtype: int64
ID de comercios con NaN en espera_en_vendor:
1392      8176278
1767    126079698
Name: id_comercio, dtype: int64
Cantidad de registros con id_comercio 8176278 o 126079698: 0
Cantidad de registros después de eliminar: 2789595
Cantidad de registros en la tabla de comercios después de eliminar: 2567


#### Joineo la tabla de PEDIDOS ORIGINAL + REPARTIDORES + COMERCIOS

In [133]:
# Unir las tablas 'df_limpio_con_repartidores' y 'comercios' basadas en la columna 'id_comercio'
df_unido = df_limpio_con_repartidores.merge(comercios[['id_comercio', 'rating', 'espera_en_vendor']], on='id_comercio', how='left')

# Verificar los primeros registros para asegurarse de que los datos se han unido correctamente 
print(df_unido.head())
print(df_unido.columns)

                                      id_pedido  id_repartidor_x  id_comercio  \
0  iSdc/QiV5eDFvPWh8CoOiQfAev9LgQFDq/hLDSP/I7U=        124406310     98613018   
1  iSdc/QiV5eDFvPWh8CoOiQfAev9LgQFDq/hLDSP/I7U=        124406310     98613018   
2  tBMHeU17rwPedjQxWV5JhFCo02OqhElPMsevJyHmxMU=         56877906    129533028   
3  mzlqOzwDoX3fv2E2uPDMIo+YgpABTzbCEWCSOJmqfk8=         70267848    121192920   
4  mzlqOzwDoX3fv2E2uPDMIo+YgpABTzbCEWCSOJmqfk8=         70267848    121192920   

              creacion_pedido        transicion  transicion_id_repartidor  \
0  2024-06-30 21:00:09.257475  courier_notified               124406310.0   
1  2024-06-30 21:00:09.257475          accepted               124406310.0   
2  2024-06-30 21:00:09.048317            queued                69517842.0   
3  2024-06-30 21:00:13.517017  courier_notified                70267848.0   
4  2024-06-30 21:00:13.517017          accepted                70267848.0   

         creacion_transicion  transicion_coordenad

#### Despues de MERGEAR todo, verifico de vuelta los NAs

In [134]:
# Verificar la cantidad de NaN en cada columna del DataFrame 'df_unido'
nas_por_columna = df_unido.isna().sum()

# Mostrar los resultados
print("Cantidad de NaN por columna en 'df_unido':")
print(nas_por_columna)


Cantidad de NaN por columna en 'df_unido':
id_pedido                             0
id_repartidor_x                       0
id_comercio                           0
creacion_pedido                       0
transicion                            0
transicion_id_repartidor              0
creacion_transicion                   0
transicion_coordenada_latitud         0
transicion_coordenada_longitud        0
tiempo_estimado_entrega               0
tiempo_aceptacion_orden               0
distancia_retiro                      0
distancia_entrega                     0
ubicacion_cliente_longitud            0
ubicacion_cliente_latitud             0
ubicacion_comercio_longitud           0
ubicacion_comercio_latitud            0
pre_accept_undispatch                 0
post_accept_undispatchs               0
pago_base                             0
pago_rendimiento                      0
pago_incentivos                       0
distancia_repartidor_comercio         0
distancia_pedido_total               

In [135]:
# Filtrar los registros que contienen NaN en cualquier columna
registros_con_nas = df_unido[df_unido.isna().any(axis=1)]

# Mostrar los registros con NaN
print("Registros con NaN en 'df_unido':")
print(registros_con_nas)


Registros con NaN en 'df_unido':
                                            id_pedido  id_repartidor_x  \
987      EoNbUYd6hfZT2AzTW5wUcScRXVSB0VhyS0MY2fJQJFU=         39676788   
3394     v/FhT7db5FkKzeBeOWXidssnUfFyDHI+15EYlKA6v4E=         43425012   
3430     v/FhT7db5FkKzeBeOWXidssnUfFyDHI+15EYlKA6v4E=         43425012   
3431     v/FhT7db5FkKzeBeOWXidssnUfFyDHI+15EYlKA6v4E=         43425012   
3725     v/FhT7db5FkKzeBeOWXidssnUfFyDHI+15EYlKA6v4E=         43425012   
...                                               ...              ...   
2789353  rCYIcLfds/qfSR/E5nWkIMWE9065YoemihKkm0vlkuo=         12902064   
2789416  rCYIcLfds/qfSR/E5nWkIMWE9065YoemihKkm0vlkuo=         12902064   
2789417  bJcvdzpm4lg7lAFYlC4pO/vdkRaoHydEhOIlHIWGOnQ=        121453500   
2789452  rCYIcLfds/qfSR/E5nWkIMWE9065YoemihKkm0vlkuo=         12902064   
2789517  bJcvdzpm4lg7lAFYlC4pO/vdkRaoHydEhOIlHIWGOnQ=        121453500   

         id_comercio             creacion_pedido        transicion  \
987     

#### Se ve que hay repartidores a los cuales se le asigna el pedido, pero que no se registran en la tabla de REPARTIDORES brindada
#### Se ve que hay restaurantes a los cuales se le asigna pedidos, pero que no se registran en la tabla de COMERCIOS brindada

#### Se decide borrar esos registros de la tabla de PEDIDOS ORIGINAL, por falta de consistencia en datos

In [136]:
# Filtrar los datos del repartidor con id_repartidor = 112040370
repartidor_filtrado = repartidores[repartidores['id_repartidor'] == 112040370]

# Mostrar los datos del repartidor
print(repartidor_filtrado)


Empty DataFrame
Columns: [id_repartidor, vehiculo, tasa_aceptacion, tasa_cancelacion, pedidos_entregados, antiguedad_semanas, tiempo_medio_entrega]
Index: []


In [137]:
# Filtrar los datos del comercio con id_comercio = 130062960
comercio_filtrado = comercios[comercios['id_comercio'] == 130062960]

# Mostrar los datos del comercio
print(comercio_filtrado)


Empty DataFrame
Columns: [id_comercio, rating, espera_en_vendor]
Index: []


In [146]:
# Eliminar los registros que tengan NaN en cualquier columna
df_unido = df_unido.dropna()

# Verificar la cantidad de registros después de eliminar
print(f"Cantidad de registros después de eliminar NaN: {len(df_unido)}")

Cantidad de registros después de eliminar NaN: 2775650


#### Unir la tabla de CLIMA

In [139]:
import pandas as pd

archivo_csv = r'C:/Users/paula/OneDrive/Documentos/Facultad/Analítica en Transporte y Logística/Base Clima.csv'
# Mostrar todas las columnas con sus respectivos valores
pd.set_option('display.max_columns', None) 
clima = pd.read_csv(archivo_csv)

print(clima.head())
print(clima.shape)

                  date event_type  temperature  precip_mm  wind  \
0  2024-05-01 00:00:00    weather         16.7       0.04  15.8   
1  2024-05-01 01:00:00    weather         16.6       0.07  15.5   
2  2024-05-01 02:00:00    weather         16.4       0.08  15.8   
3  2024-05-01 03:00:00    weather         16.3       0.02  17.6   
4  2024-05-01 04:00:00    weather         16.2       0.00  17.6   

              condition  
0  Patchy rain possible  
1  Patchy rain possible  
2  Patchy rain possible  
3  Patchy rain possible  
4                Cloudy  
(2136, 6)


#### Me fijo el rango de fechas con el que estamos trabajando y si estan los datos de clima de estos rangos de fecha-hora

In [147]:
# Asegurarse de que la columna 'creacion_transicion' está en formato datetime
df_unido['creacion_transicion'] = pd.to_datetime(df_unido['creacion_transicion'], errors='coerce')

# Obtener la primera y la última fecha de la columna 'creacion_transicion'
primera_fecha = df_unido['creacion_transicion'].min()
ultima_fecha = df_unido['creacion_transicion'].max()

# Mostrar el rango de fechas
print(f"Primera fecha de 'creacion_transicion': {primera_fecha}")
print(f"Última fecha de 'creacion_transicion': {ultima_fecha}")

# Asegurarse de que la columna 'date' está en formato datetime
clima['date'] = pd.to_datetime(clima['date'], errors='coerce')

# Obtener la primera y la última fecha de la columna 'date'
primera_fecha2 = clima['date'].min()
ultima_fecha2 = clima['date'].max()

# Mostrar el rango de fechas
print(f"Primera fecha de 'date': {primera_fecha2}")
print(f"Última fecha de 'date': {ultima_fecha2}")


Primera fecha de 'creacion_transicion': 2024-06-30 21:02:10.867048
Última fecha de 'creacion_transicion': 2024-07-31 21:59:44.124501
Primera fecha de 'date': 2024-05-01 00:00:00
Última fecha de 'date': 2024-07-31 23:00:00


In [148]:
# Verificar la cantidad de NaN en cada columna del DataFrame 'clima'
nas_por_columna = clima.isna().sum()

# Mostrar los resultados
print("Cantidad de NaN por columna en 'clima':")
print(nas_por_columna)

Cantidad de NaN por columna en 'clima':
date           0
event_type     0
temperature    0
precip_mm      0
wind           0
condition      0
dtype: int64


#### PEDIDOS + REPARTIDORES + COMERCIOS + CLIMA

In [149]:
# Asegurarse de que la columna 'creacion_transicion' está en formato datetime
df_unido['creacion_transicion'] = pd.to_datetime(df_unido['creacion_transicion'], errors='coerce')

# Redondear la columna 'creacion_transicion' a la hora más cercana
df_unido['creacion_transicion_redondeada'] = df_unido['creacion_transicion'].dt.floor('H')

# Asegurarse de que la columna 'date' en 'clima' está en formato datetime
clima['date'] = pd.to_datetime(clima['date'], errors='coerce')

# Realizar un merge entre 'df_unido' y 'clima' basado en la columna redondeada de 'creacion_transicion' y 'date'
df_final = df_unido.merge(clima[['date', 'event_type', 'temperature', 'precip_mm', 'wind', 'condition']], 
                          left_on='creacion_transicion_redondeada', right_on='date', how='left')

# Verificar los primeros registros para asegurarse de que los datos se han unido correctamente
print(df_final.head())
print(df_final.columns)


                                      id_pedido  id_repartidor_x  id_comercio  \
0  iSdc/QiV5eDFvPWh8CoOiQfAev9LgQFDq/hLDSP/I7U=        124406310     98613018   
1  iSdc/QiV5eDFvPWh8CoOiQfAev9LgQFDq/hLDSP/I7U=        124406310     98613018   
2  tBMHeU17rwPedjQxWV5JhFCo02OqhElPMsevJyHmxMU=         56877906    129533028   
3  mzlqOzwDoX3fv2E2uPDMIo+YgpABTzbCEWCSOJmqfk8=         70267848    121192920   
4  mzlqOzwDoX3fv2E2uPDMIo+YgpABTzbCEWCSOJmqfk8=         70267848    121192920   

             creacion_pedido        transicion  transicion_id_repartidor  \
0 2024-06-30 21:00:09.257475  courier_notified               124406310.0   
1 2024-06-30 21:00:09.257475          accepted               124406310.0   
2 2024-06-30 21:00:09.048317            queued                69517842.0   
3 2024-06-30 21:00:13.517017  courier_notified                70267848.0   
4 2024-06-30 21:00:13.517017          accepted                70267848.0   

         creacion_transicion  transicion_coordenada_lati

In [151]:
# Verificar la cantidad de NaN en cada columna del DataFrame 'clima'
nas_por_columna = df_final.isna().sum()

# Mostrar los resultados
print("Cantidad de NaN por columna df_final:")
print(nas_por_columna)

Cantidad de NaN por columna df_final:
id_pedido                              0
id_repartidor_x                        0
id_comercio                            0
creacion_pedido                        0
transicion                             0
transicion_id_repartidor               0
creacion_transicion                    0
transicion_coordenada_latitud          0
transicion_coordenada_longitud         0
tiempo_estimado_entrega                0
tiempo_aceptacion_orden                0
distancia_retiro                       0
distancia_entrega                      0
ubicacion_cliente_longitud             0
ubicacion_cliente_latitud              0
ubicacion_comercio_longitud            0
ubicacion_comercio_latitud             0
pre_accept_undispatch                  0
post_accept_undispatchs                0
pago_base                              0
pago_rendimiento                       0
pago_incentivos                        0
distancia_repartidor_comercio          0
distancia_pedido_to

In [156]:
# Filtrar los registros donde la columna 'date' sea NaN
registros_con_nan_date = df_final[df_final['date'].isna()]

# Seleccionar las columnas que deseas mostrar
resultado = registros_con_nan_date[['creacion_transicion', 'creacion_transicion_redondeada', 'date']]

# Mostrar los registros filtrados
print(resultado)


               creacion_transicion creacion_transicion_redondeada date
1563782 2024-07-18 00:00:01.636033            2024-07-18 00:00:00  NaT
1563783 2024-07-18 00:00:02.374142            2024-07-18 00:00:00  NaT
1563784 2024-07-18 00:00:02.424145            2024-07-18 00:00:00  NaT
1563785 2024-07-18 00:00:02.747836            2024-07-18 00:00:00  NaT
1563786 2024-07-18 00:00:03.497134            2024-07-18 00:00:00  NaT
...                            ...                            ...  ...
2005348 2024-07-22 23:59:52.691545            2024-07-22 23:00:00  NaT
2005349 2024-07-22 23:59:53.569442            2024-07-22 23:00:00  NaT
2005350 2024-07-22 23:59:55.021332            2024-07-22 23:00:00  NaT
2005351 2024-07-22 23:59:58.434009            2024-07-22 23:00:00  NaT
2005352 2024-07-22 23:59:58.492821            2024-07-22 23:00:00  NaT

[181586 rows x 3 columns]


In [154]:
# Definir el rango de fechas
fecha_inicio = '2024-06-30 21:00:00'
fecha_fin = '2024-07-31 21:00:00'

# Filtrar los registros donde la columna 'date' esté dentro del rango de fechas
registros_rango_fechas = clima[(clima['date'] >= fecha_inicio) & (clima['date'] <= fecha_fin)]

# Mostrar los registros correspondientes
print("Registros en 'clima' entre 2024-06-30 21:00:00 y 2024-07-31 21:00:00:")
print(registros_rango_fechas)


Registros en 'clima' entre 2024-06-30 21:00:00 y 2024-07-31 21:00:00:
                    date event_type  temperature  precip_mm  wind  \
1437 2024-06-30 21:00:00    weather         10.8        0.0  23.4   
1438 2024-06-30 22:00:00    weather         10.7        0.0  24.1   
1439 2024-06-30 23:00:00    weather         10.4        0.0  23.8   
1440 2024-07-01 00:00:00    weather         10.1        0.0  21.2   
1441 2024-07-01 01:00:00    weather         10.0        0.0  20.2   
...                  ...        ...          ...        ...   ...   
2129 2024-07-31 17:00:00    weather         15.6        0.0  13.7   
2130 2024-07-31 18:00:00    weather         14.4        0.0  13.0   
2131 2024-07-31 19:00:00    weather         14.1        0.0  13.7   
2132 2024-07-31 20:00:00    weather         14.3        0.0  11.9   
2133 2024-07-31 21:00:00    weather         14.5        0.0  10.8   

          condition  
1437       Overcast  
1438       Overcast  
1439       Overcast  
1440  Partly 

In [155]:
import pandas as pd

# Asegúrate de que 'registros_rango_fechas' es un DataFrame
# Suponiendo que 'registros_rango_fechas' ya ha sido filtrado como en los pasos anteriores

# Exportar a un archivo Excel
registros_rango_fechas.to_excel('registros_clima_rango_fechas.xlsx', index=False)

print("Archivo Excel exportado exitosamente.")


Archivo Excel exportado exitosamente.


#### Pudimos corroborar que las fechas del 18 y 22/7/2024 no tiene datos del clima

#### Borramos los registros de esas fechas

In [158]:
# Eliminar los registros que tienen NaN en la columna 'date'
df_final = df_final.dropna(subset=['date'])

# Verificar si hay NaN en la tabla después de eliminar los registros
nas_por_columna = df_final.isna().sum()

# Mostrar los resultados
print("Cantidad de NaN por columna en 'df_final_sin_nas':")
print(nas_por_columna)


Cantidad de NaN por columna en 'df_final_sin_nas':
id_pedido                         0
id_repartidor_x                   0
id_comercio                       0
creacion_pedido                   0
transicion                        0
transicion_id_repartidor          0
creacion_transicion               0
transicion_coordenada_latitud     0
transicion_coordenada_longitud    0
tiempo_estimado_entrega           0
tiempo_aceptacion_orden           0
distancia_retiro                  0
distancia_entrega                 0
ubicacion_cliente_longitud        0
ubicacion_cliente_latitud         0
ubicacion_comercio_longitud       0
ubicacion_comercio_latitud        0
pre_accept_undispatch             0
post_accept_undispatchs           0
pago_base                         0
pago_rendimiento                  0
pago_incentivos                   0
distancia_repartidor_comercio     0
distancia_pedido_total            0
aceptado                          0
tiempo_hasta_entrega              0
id_repartidor

#### Revisamos la tabla df_final

In [159]:
print(df_final.head())
print(df_final.shape)
print(df_final.columns)

                                      id_pedido  id_repartidor_x  id_comercio  \
0  iSdc/QiV5eDFvPWh8CoOiQfAev9LgQFDq/hLDSP/I7U=        124406310     98613018   
1  iSdc/QiV5eDFvPWh8CoOiQfAev9LgQFDq/hLDSP/I7U=        124406310     98613018   
2  tBMHeU17rwPedjQxWV5JhFCo02OqhElPMsevJyHmxMU=         56877906    129533028   
3  mzlqOzwDoX3fv2E2uPDMIo+YgpABTzbCEWCSOJmqfk8=         70267848    121192920   
4  mzlqOzwDoX3fv2E2uPDMIo+YgpABTzbCEWCSOJmqfk8=         70267848    121192920   

             creacion_pedido        transicion  transicion_id_repartidor  \
0 2024-06-30 21:00:09.257475  courier_notified               124406310.0   
1 2024-06-30 21:00:09.257475          accepted               124406310.0   
2 2024-06-30 21:00:09.048317            queued                69517842.0   
3 2024-06-30 21:00:13.517017  courier_notified                70267848.0   
4 2024-06-30 21:00:13.517017          accepted                70267848.0   

         creacion_transicion  transicion_coordenada_lati

In [162]:
import pandas as pd

# Filtrar los registros donde 'id_pedido' es igual al valor especificado
id_pedido_filtrado = df_final[df_final['id_pedido'] == 'tBMHeU17rwPedjQxWV5JhFCo02OqhElPMsevJyHmxMU=']

# Exportar los registros filtrados a un archivo Excel
id_pedido_filtrado.to_excel('id_pedido_filtrado.xlsx', index=False)

# Mostrar los primeros registros para confirmar
print(id_pedido_filtrado.head())
print("Archivo exportado exitosamente.")


                                         id_pedido  id_repartidor_x  \
2     tBMHeU17rwPedjQxWV5JhFCo02OqhElPMsevJyHmxMU=         56877906   
585   tBMHeU17rwPedjQxWV5JhFCo02OqhElPMsevJyHmxMU=         56877906   
590   tBMHeU17rwPedjQxWV5JhFCo02OqhElPMsevJyHmxMU=         56877906   
690   tBMHeU17rwPedjQxWV5JhFCo02OqhElPMsevJyHmxMU=         56877906   
1485  tBMHeU17rwPedjQxWV5JhFCo02OqhElPMsevJyHmxMU=         56877906   

      id_comercio            creacion_pedido        transicion  \
2       129533028 2024-06-30 21:00:09.048317            queued   
585     129533028 2024-06-30 21:00:09.048317  courier_notified   
590     129533028 2024-06-30 21:00:09.048317          accepted   
690     129533028 2024-06-30 21:00:09.048317       near_pickup   
1485    129533028 2024-06-30 21:00:09.048317         picked_up   

      transicion_id_repartidor        creacion_transicion  \
2                   69517842.0 2024-06-30 21:02:28.411115   
585                 56877906.0 2024-06-30 21:19:06.088

#### Categorizar el tipo de clima

In [163]:
# Ver las distintas categorías de la columna 'condition'
categorias_condition = df_final['condition'].unique()

# Mostrar los resultados
print("Categorías únicas en la columna 'condition':")
print(categorias_condition)

Categorías únicas en la columna 'condition':
['Overcast' 'Partly cloudy' 'Clear' 'Sunny' 'Cloudy'
 'Patchy rain possible' 'Light drizzle' 'Patchy light drizzle'
 'Light rain' 'Mist']


In [165]:
# Crear una función para asignar el valor a la nueva columna 'tipo_condition'
def asignar_tipo_condition(condition):
    if condition in ['Sunny', 'Clear', 'Partly cloudy', 'Overcast', 'Cloudy', 'Mist']:
        return 1
    elif condition in ['Patchy light drizzle', 'Light drizzle']:
        return 2
    elif condition in ['Light rain', 'Patchy rain possible']:
        return 3
    else:
        return None  # Por si aparecen otros valores no contemplados

# Aplicar la función a la columna 'condition' para crear la nueva columna 'tipo_condition'
df_final['tipo_condition'] = df_final['condition'].apply(asignar_tipo_condition)


In [167]:
# Eliminar las columnas 'id_repartidor_y' y 'event_type' de df_final
df_final = df_final.drop(columns=['id_repartidor_y', 'event_type'])

#### Exporto la base LIMPIA

In [168]:
# Exportar el DataFrame df_final a un archivo CSV
df_final.to_csv('df_limpio_PEYA.csv', index=False)

# Confirmación de la exportación
print("Archivo 'df_limpio_PEYA.csv' exportado exitosamente.")


Archivo 'df_limpio_PEYA.csv' exportado exitosamente.
